<a href="https://colab.research.google.com/github/yugan243/AI-Task-Manager-Agent/blob/main/Day0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Qdrant Essentials: Day 0: Creating First Collection

### Step 1: Install the Qdrant Client

In [1]:
! pip install -q qdrant-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.4/390.4 kB 7.6 MB/s eta 0:00:00


### Step 2: Import Required Libraries

In [2]:
from qdrant_client import QdrantClient, models

### Step 3: Connect to Qdrant Cloud

In [3]:
import os
from google.colab import userdata

client = QdrantClient("https://fa13d3f3-6758-4e1a-9777-f66dc4f300c2.us-east4-0.gcp.cloud.qdrant.io", api_key=userdata.get('api-key'))


### Step 4: Create a Collection

A Collection in Qdrant is analogous to the table in relational databases. It's a container for storing vectors and its metadata.
When creating a collection, we need to specify:
1. Name: A unique identifier for the collection
2. Vector configuration
3. Size: The dimentionality of vectors
4. Distance metric: the method used to measure similarity between vectors

In [4]:
collection_name = "my_first_collection"

# Create the collection with the specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=4, # Dimentionality of vectors
        distance=models.Distance.COSINE # Distance metric for similarity search
    )
)

True

### Step 5: Verify the Collection Creation

In [5]:
collections = client.get_collections()
print("Existing collections: ", collections)

Existing collections:  collections=[CollectionDescription(name='my_first_collection'), CollectionDescription(name='points'), CollectionDescription(name='inference_test')]


### Step 6: Insert the points to the collection

In [7]:
# Define the vecetors to be inserted
points = [
    models.PointStruct(
        id=1,
        vector=[0.1, 0.2, 0.3, 0.4],
        payload={"category": "example"}
    ),
    models.PointStruct(
        id=2,
        vector=[0.2, 0.3, 0.4, 0.5],
        payload={"category": "demo"}
    )
]

# Insert vectors to the collection
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

### 7. Retrieving collection details


In [8]:
collection_info = client.get_collection(collection_name)
print("Collection info: ", collection_info)

Collection info:  status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> warnings=None indexed_vectors_count=0 points_count=2 segments_count=2 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=4, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, read_fan_out_delay_ms=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None, inline_storage=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=10000, flush_interval_sec=5, max_optimization_threads=None, prevent_unoptimized=None), wal_

### 8. Running a Similarity Search

In [9]:
query_vector = [0.08, 0.14, 0.33, 0.28]

search_results = client.query_points(
    collection_name=collection_name,
    query=query_vector,
    limit=1
)

print("Search results: ", search_results)

Search results:  points=[ScoredPoint(id=1, version=1, score=0.97642946, payload={'category': 'example'}, vector=None, shard_key=None, order_value=None)]


### Step 9: Filtered Search

In [13]:
search_filter = models.Filter(
    must=[
        models.FieldCondition(
            key="category",
            match=models.MatchValue(value="example")
        )
    ]
)

filtered_results = client.query_points(
    collection_name=collection_name,
    query=query_vector,
    query_filter=search_filter,
    limit=1
)

print("Filtered search results after indexing: ", filtered_results)

UnexpectedResponse: Unexpected Response: 400 (Bad Request)
Raw response content:
b'{"status":{"error":"Bad request: Index required but not found for \\"category\\" of one of the following types: [keyword]. Help: Create an index for this key or use a different filter."},"time":0.000 ...'